In [ ]:
from huggingface_hub import InferenceClient

In [ ]:
MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B"
TOKEN = ''

In [ ]:
client = InferenceClient(MODEL_NAME, token=TOKEN)

In [ ]:
def hse_assistant(user_message):
  messages = [
      {
          "role": "system", # оставили тем же, что и в прошлый раз
          "content": ("Ты помощник студента. "
          "Ты должен составлять варианты тестовых заданий, конспекты и отвечать на вопросы по теме заданной пользователем. "
          "Генерируй только тестовые задания, конспекты и ответы на вопросы пользователя на русском языке, без вводных фраз. "
          "Используй заданные функции для того, чтобы помогать студентам. "),
      },
      {
          "role": "user",
          "content": user_message,
      },
  ]
  tools = [
      {
          "type": "function",
          "function": {
              "name": "test_questions_generator", # Автор: Яна Хлусова
              "description": "Генерирует тестовые вопросы по теме, заданной пользователем. ",
              "parameters": {
                  "type": "object",
                  "properties": {
                        "test": {
                            "type": "string",
                            "description": "Сгенерированный тест. "
                            }
                  },
                  "required": ["test"] # у меня все кратенько, чтобы токены не съедались
                  # плюс заметила, при попытке кастомизировать тесты, они перестают генериться
                  # лучше дать в инструкцию к юзер промпту, чтобы студенты сами писали, сколько хотят вопросов и вариантов ответа
              }
          }
      },
      {
            "type": "function",
            "function": {
                "name": "conspect_maker", # Автор: Соня Шевцова
                "description": "Генерирует конспект по теме, заданной пользователем. ",
                "parameters": {
                    "type": "object",
                    "properties": {
                         "topic": {
                            "type": "string",
                            "description": "Тема, по которой нужно создать конспект. "
                            },
                         "summary_type": {
                            "type": "string",
                            "enum": ["подробный", "краткий"],
                            "description": "Тип конспекта: 'подробный' с детальными объяснениями или 'краткий' с основными тезисами."
                            },
                         "num_points": {
                            "type": "integer",
                            "description": "Желаемое количество пунктов в итоговом конспекте. "
                            },
                          "summary": {
                            "type": "string",
                            "description": "Сгенерированный конспект по заданной теме."
                        }
                    },
                    "required": ["topic", "summary_type", "num_points", "summary"]
                }
            }
       },
       {
          "type": "function",
            "function": {
                "name": "review_assignment", # Автор: Катя Костина
                "description": ("# Роль Ты помощник студента. "
                "# Задача Ты должен проверять выполненные задания, давать обратную связь. "
                "Анализируй ответы и давай развернутую обратную связь: "
                "- Что студент написал правильно? "
                "- Что в ответе стдента требует уточнения или доработки? "
                "- Как можно улучшить ответ, чтобы соответствовать заданиию "
                "Учитывай текст задания при составлении вопросов и оценке ответов и рекомендуй дополнительные материалы из базы знаний. "),
                "parameters": {
                    "type": "object",
                    "properties": {
                        "feedback": {
                            "type": "string",
                            "description": "Обратная связь по выполненной работе: корректность, сильные и слабые стороны, рекомендации.",
                        },
                        "recommended_materials": {
                            "type": "array",
                            "items": {
                                "type": "string",
                                "description": "Список рекомендуемых материалов из базы знаний для улучшения понимания темы."
                            }
                        }
                    },
                    "required": ["feedback", "recommended_materials"]
                }
            }
      }
  ]

  response = client.chat_completion(
      model=MODEL_NAME,
      messages=messages,
      tools=tools,
      tool_choice="auto",
      max_tokens=1000,
      temperature=0.3,
      top_p=0.1,
    )

  return response.choices[0].message.tool_calls[0].function.arguments

In [ ]:
user_message = "Проверь мой словообразовательный разбоор слова: Соклетие «— клетка, база — клетк. С усечением основы (-клет-), формант — префикс со-, суффикс -ий-.Сл-й тип: произв-я основа су с усечением + префикс со- + суффикс -ий-, С.з. — ‘соединение’, созвездие, Способ — приставочно-суффиксальный."
#"Проверь мое решение задачи по математике. Задача: В ящике имеются белый и черный шары. Наугад извлекается шар. Если извлеченный шар белый, его возвращают в ящик и добавляют туда еще два белых шара. Какова вероятность извлечь черный шар во второй раз. Мое Решение. Так как не указано, что черный шар возвращают на место, будем считать, что его не возвращают.Предположим, что в 1 раз мы вытащили белый шар. Тогда в ящике теперь 4 шара: 3 белых и 1 черный.Подсчитаем вероятности элементарных исходов: вытащить белый шар = 1/2 вытащить черный шар = 1/4 Так как события происходят одновременно, их нужно умножить 1/2 * 1/4 = 1/8 Ответ: вероятность извлечь черный шар во второй раз = 1/8"
# "Создай подробный конспект по теме 'линейная регрессия' на 8 пунктов. "
# 'Составь 5 тестовых вопросов по теме Иерархия Хомского. '

response = hse_assistant(user_message)

In [ ]:
print(response)
# примечательно, что при проверке заданий в рекомендованные материалы попадают болванки сайтов, т.к. база знаний пока не подключена

{'recommended_materials': ['https://example.com/word-formation', 'https://example.com/prefixes-suffixes'], 'feedback': "Ваш разбор слова 'Соклетие' содержит правильные элементы, но требует уточнения. Вы правильно определили основу '-клет-' и суффикс '-ий-'. Однако префикс 'со-' в данном случае не является приставкой, а является частью корня. Следует также отметить, что слово 'Соклетие' образовано от 'клеть' с помощью приставки 'со-' и суффикса '-ие'. Рекомендуется пересмотреть тип словообразования и уточнить роль приставки и суффикса."}


In [ ]:
dataset["user_prompts"].append(user_message)
dataset["results"].append(response)

In [ ]:
import pandas as pd

df = pd.DataFrame(dataset)
df.head()

,user_prompts,results
0,Создай подробный конспект по теме 'линейная ре...,{'summary': '1. Основные понятия линейной регр...
1,Составь 5 тестовых вопросов по теме Иерархия Х...,{'test': '1. Какие уровни включает иерархия Хо...
2,Проверь мое решение задачи по математике. Зада...,{'recommended_materials': ['https://example.co...
3,Проверь мой словообразовательный разбоор слова...,{'recommended_materials': ['https://example.co...


In [ ]:
df.to_csv('kostina_shevtsova_khlusova.csv')